In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# [TR-013] Spin alignment

<!-- cspell:ignore phasespace -->
```{autolink-skip}
```

In [ ]:
%pip install -q git+https://github.com/ComPWA/ampform@98de70f pandas==1.4.1 qrules[viz]==0.9.7 tensorwaves[jax,pwa]==0.4.2

:::{note}

This report is a continuation of [ampform#245](https://ampform--245.org.readthedocs.build/en/245/usage/helicity/spin-alignment.html).

:::

<!-- cspell:ignore Marangotto Multibody lstrip rlcrl rstrip -->

In {ref}`report/013:Distribution without alignment`, we attempt to  attempts to reproduce the distributions shown in [Figure 3](https://downloads.hindawi.com/journals/ahep/2020/6674595.pdf#page=10).[^marangotto] Next, in {ref}`report/013:Spin alignment sum`, we attempt to reproduce the distribution shown in [Figure 2](https://downloads.hindawi.com/journals/ahep/2020/6674595.pdf#page=9) with the new mechanisms introduced in [ampform#245](https://ampform--245.org.readthedocs.build/en/245/usage/helicity/spin-alignment.html).

[^marangotto]: D. Marangotto. Helicity Amplitudes for Generic Multibody Particle Decays Featuring Multiple Decay Chains. _Advances in High Energy Physics_, 2020:1–15, December 2020. [doi:10.1155/2020/6674595](https://doi.org/10.1155/2020/6674595).

In [ ]:
import logging
import warnings

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

## Phase space sample

In [ ]:
import qrules

PDG = qrules.load_pdg()

In [ ]:
from tensorwaves.data import (
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
)

phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=PDG["Lambda(c)+"].mass,
    final_state_masses={
        0: PDG["p"].mass,
        1: PDG["K-"].mass,
        2: PDG["pi+"].mass,
    },
)
rng = TFUniformRealNumberGenerator(seed=0)
phsp_momenta = phsp_generator.generate(100_000, rng)

## Distribution without alignment

### Create amplitude model

In [ ]:
from qrules.particle import ParticleCollection, create_particle

particle_db = ParticleCollection()
particle_db.add(PDG["Lambda(c)+"])
particle_db.add(PDG["p"])
particle_db.add(PDG["K-"])
particle_db.add(PDG["pi+"])

particle_db.add(
    create_particle(
        PDG["K*(892)0"],
        name="K*",
        latex="K^*",
    )
)
particle_db.add(
    create_particle(
        PDG["Lambda(1405)"],
        name="Lambda*",
        latex=R"\Lambda^*",
    )
)
particle_db.add(
    create_particle(
        PDG["Delta(1232)++"],
        name="Delta*++",
        latex=R"\Delta^*",
    )
)

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("Lambda(c)+", [-0.5, +0.5]),
    final_state=["p", "K-", "pi+"],
    formalism="helicity",
    particle_db=particle_db,
)

In [ ]:
import graphviz

n = len(reaction.transitions)
for t in reaction.transitions[:: n // 3]:
    dot = qrules.io.asdot([t], collapse_graphs=True, size=3.5)
    graph = graphviz.Source(dot)
    display(graph)

Amplitude model formulated following [Appendix C](https://downloads.hindawi.com/journals/ahep/2020/6674595.pdf#page=13):

In [ ]:
import ampform
from ampform.dynamics.builder import RelativisticBreitWignerBuilder

builder = ampform.get_builder(reaction)
builder.stable_final_state_ids = list(reaction.final_state)
builder.scalar_initial_state_mass = True
bw_builder = RelativisticBreitWignerBuilder()
for name in reaction.get_intermediate_particles().names:
    builder.set_dynamics(name, bw_builder)
model = builder.formulate()

In [ ]:
import sympy as sp
from IPython.display import Math, display

sub_exprs = {
    sp.Symbol(name): expr
    for name, expr in model.components.items()
    if name.startswith("I")
}

top_expr = model.expression.xreplace({e: s for s, e in sub_exprs.items()})
latex = sp.multiline_latex(
    sp.Symbol("I"), top_expr, terms_per_line=4, environment="eqnarray"
)
display(Math(latex))

for symbol, expr in sub_exprs.items():
    amp = expr.args[0].args[0]
    latex = sp.multiline_latex(symbol, amp, environment="eqnarray")
    display(Math(latex))

Importing the parameter values given by [Table 1](https://downloads.hindawi.com/journals/ahep/2020/6674595.pdf#page=13):

In [ ]:
# K*
model.parameter_defaults[1] = 1
model.parameter_defaults[3] = 0.5 + 0.5j
model.parameter_defaults[2] = 1j
model.parameter_defaults[1] = -0.5 - 0.5j
model.parameter_defaults[15] = 0.9  # GeV
model.parameter_defaults[8] = 0.2  # GeV

# Lambda*
model.parameter_defaults[7] = 1j
model.parameter_defaults[6] = 0.8 - 0.4j
model.parameter_defaults[17] = 1.6  # GeV
model.parameter_defaults[10] = 0.2  # GeV

# Delta*
model.parameter_defaults[5] = 0.6 - 0.4j
model.parameter_defaults[4] = 0.1j
model.parameter_defaults[16] = 1.4  # GeV
model.parameter_defaults[9] = 0.2  # GeV

In [ ]:
latex = R"\begin{array}{lc}" + "\n"
for i in [1, 3, 2, 1, 15, 8, 7, 6, 17, 10, 5, 4, 16, 9]:
    p, v = list(model.parameter_defaults.items())[i]
    value = str(v).lstrip("(").rstrip(")").replace("j", "i")
    latex += Rf"  {sp.latex(p)} & {value} \\" + "\n"
latex += R"\end{array}"
Math(latex)

### Generate data

In [ ]:
from tensorwaves.data import SympyDataTransformer

helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)
phsp = helicity_transformer(phsp_momenta)
phsp = {k: v.real for k, v in phsp.items()}

In [ ]:
import pandas as pd

pd.DataFrame(phsp).round(3)

In [ ]:
full_expression = model.expression.doit()
substituted_expression = full_expression.xreplace(model.parameter_defaults)

In [ ]:
from tensorwaves.function.sympy import create_function

intensity_func = create_function(substituted_expression, backend="jax")

In [ ]:
import numpy as np

intensities = np.array(intensity_func(phsp).real)
intensities.round(4)

In [ ]:
def compute_sub_intensities(resonance_name: str):
    parameter_values = {}
    for symbol, value in model.parameter_defaults.items():
        if resonance_name in symbol.name:
            parameter_values[symbol] = value
        else:
            parameter_values[symbol] = 0
    sub_expression = full_expression.subs(parameter_values)
    sub_intensity = create_function(sub_expression, backend="jax")
    return np.array(sub_intensity(phsp).real)


intensities_k = compute_sub_intensities("K^*")
intensities_delta = compute_sub_intensities("Delta^*")
intensities_lambda = compute_sub_intensities("Lambda^*")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 5))
hist_kwargs = dict(
    bins=80,
    histtype="step",
)

for x in ax.flatten():
    x.set_yticks([])

ax[0, 0].set_xlabel("$m^2(pK^-)$ [GeV$^2/c^4$]")
ax[0, 1].set_xlabel(R"$m^2(K^-\pi^+)$ [GeV$^2/c^4$]")
ax[0, 2].set_xlabel(R"$m^2(p\pi^+)$ [GeV$^2/c^4$]")
ax[1, 0].set_xlabel(R"$\cos\theta(p)$")
ax[1, 1].set_xlabel(R"$\phi(p)$")
ax[1, 2].set_xlabel(R"$\chi$")

for x, xticks in {
    ax[0, 0]: [2, 2.5, 3, 3.5, 4, 4.5],
    ax[0, 1]: [0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2],
    ax[0, 2]: [1, 1.5, 2, 2.5, 3],
    ax[1, 0]: [-1, -0.5, 0, 0.5, 1],
    ax[1, 1]: [-3, -2, -1, 0, 1, 2, 3],
}.items():
    x.set_xticks(xticks)
    x.set_xticklabels(xticks)

for weights, color, label in [
    (intensities, "red", "Model"),
    (intensities_k, "orange", R"$K^*\to\,K^{^-}\pi^+$"),
    (intensities_delta, "brown", R"$\Delta^{*^{++}} \to\,p\pi^+$"),
    (intensities_lambda, "purple", R"$\Lambda^* \to\,p K^{^-}$"),
]:
    kwargs = dict(weights=weights, color=color, **hist_kwargs)
    ax[0, 0].hist(np.array(phsp["m_01"] ** 2), **kwargs)
    ax[0, 1].hist(np.array(phsp["m_12"] ** 2), **kwargs)
    ax[0, 2].hist(np.array(phsp["m_02"] ** 2), **kwargs)
    ax[1, 0].hist(np.array(np.cos(phsp["theta_01"])), **kwargs)
    ax[1, 1].hist(np.array(phsp["phi_01"]), **kwargs, label=label)

ax[1, 2].remove()
handles, labels = ax[1, 1].get_legend_handles_labels()
fig.legend(handles, labels, loc="lower right")

ax[0, 2].set_xlim(1, 3.4)
ax[1, 0].set_xlim(-1, +1)
ax[1, 1].set_xlim(-np.pi, +np.pi)

fig.tight_layout()

plt.show()

Compare with [Figure 3](https://downloads.hindawi.com/journals/ahep/2020/6674595.pdf#page=10).

## Spin alignment sum

:::{note}

Need to implement [ampform#245](https://ampform--245.org.readthedocs.build/en/245/usage/helicity/spin-alignment.html) in the {class}`~ampform.helicity.HelicityAmplitudeBuilder`.

:::